In [1]:
import requests, spotipy, scipy.spatial.distance, pandas as pd, numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import lil_matrix 
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
# Authentication credentials
client_id = '67dd441a7f8249ecb919eb72ea11f460' 
client_secret = '3a521e0f708f44fe8625f92cfeaef485'
market = 'US'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

seed_tracks = ["7iQXYTyuG13aoeHxGG28Nh"] # PERRO NEGRO
seed_genres = ["latin"]
seed_artists = [
    "4q3ewBCX7sLwd24euuV69X", # bad bunny
    ] 

rec_map = []

rec_map = sp.recommendations(seed_artists=seed_artists,seed_genres=seed_genres,seed_tracks=seed_tracks,limit=50)

In [4]:
tracks_data = {}

for track in rec_map['tracks']:
        # print(track['id'])
        
        track_id = track['id']
        track_name = track['name']
        artist_id = track['artists'][0]['id']
        track_data = {
            'artist_id': artist_id,
            'track_id': track_id,
            'track_name': track_name
        }
        
        tracks_data[track_id] = track_data

# appending PERRO NEGRO IDs to dict
tracks_data['7iQXYTyuG13aoeHxGG28Nh'] = {
    'artist_id' : '4q3ewBCX7sLwd24euuV69X',
    'track_id' : '7iQXYTyuG13aoeHxGG28Nh',
    'track_name' : 'PERRO NEGRO'
}

print(tracks_data)

# recs_df = pd.DataFrame(tracks_data, columns=['artist_id', 'track_id', 'track_name'])


{'3GaGWoU1KfVqgl7vnRHP55': {'artist_id': '5TYxZTjIPqKM8K8NuP9woO', 'track_id': '3GaGWoU1KfVqgl7vnRHP55', 'track_name': 'Demasiadas Mujeres'}, '4sDpxaulErbJ9NBaJDC7cJ': {'artist_id': '52iwsT98xCoGgiGntTiR7K', 'track_id': '4sDpxaulErbJ9NBaJDC7cJ', 'track_name': 'Buenas'}, '7aJP1XcWcSU9w0eqSvaeZR': {'artist_id': '7eLcDZDYHXZCebtQmVFL25', 'track_id': '7aJP1XcWcSU9w0eqSvaeZR', 'track_name': 'Lo Malo - Operación Triunfo 2017'}, '7q3NthaBqFI6s0WnAlnIvq': {'artist_id': '1TtXnWcUs0FCkaZDPGYHdf', 'track_id': '7q3NthaBqFI6s0WnAlnIvq', 'track_name': 'Velitas'}, '5ubqSAc9LxUS2CsMHcA4kF': {'artist_id': '6IE6z7DcZIT4Ml3Fh5Ivch', 'track_id': '5ubqSAc9LxUS2CsMHcA4kF', 'track_name': 'Como La Flor'}, '1eG68fWLXO7IphzE4GOeJf': {'artist_id': '1lgtR3WlcFxEy6yPoOh0J2', 'track_id': '1eG68fWLXO7IphzE4GOeJf', 'track_name': 'Big Booty'}, '72O5JOfMVbbJVYD9DOCWMi': {'artist_id': '07YUOmWljBTXwIseAUd9TW', 'track_id': '72O5JOfMVbbJVYD9DOCWMi', 'track_name': 'Energía Bacana'}, '332fAqelC3Uik91QPxPcB8': {'artist_id': 

In [5]:
audio_features = {}

for idx in tracks_data:
        
    audio_req = sp.audio_features(idx)[0]

    track_id = audio_req['id']
    audio = audio_req
    
    audio_attributes = {
        'danceability' : audio['danceability'],
        'energy' : audio['energy'],
        'tempo' : audio['tempo'],
        'speechines' : audio['speechiness'],
        'valence' : audio['valence']
    }
    audio_features[track_id] = audio_attributes

print(audio_features)

{'3GaGWoU1KfVqgl7vnRHP55': {'danceability': 0.658, 'energy': 0.453, 'tempo': 126.043, 'speechines': 0.39, 'valence': 0.358}, '4sDpxaulErbJ9NBaJDC7cJ': {'danceability': 0.625, 'energy': 0.755, 'tempo': 87.948, 'speechines': 0.0724, 'valence': 0.429}, '7aJP1XcWcSU9w0eqSvaeZR': {'danceability': 0.766, 'energy': 0.737, 'tempo': 95.011, 'speechines': 0.0634, 'valence': 0.964}, '7q3NthaBqFI6s0WnAlnIvq': {'danceability': 0.819, 'energy': 0.596, 'tempo': 165.951, 'speechines': 0.318, 'valence': 0.622}, '5ubqSAc9LxUS2CsMHcA4kF': {'danceability': 0.632, 'energy': 0.75, 'tempo': 91.17, 'speechines': 0.0358, 'valence': 0.87}, '1eG68fWLXO7IphzE4GOeJf': {'danceability': 0.907, 'energy': 0.551, 'tempo': 152.031, 'speechines': 0.237, 'valence': 0.644}, '72O5JOfMVbbJVYD9DOCWMi': {'danceability': 0.756, 'energy': 0.739, 'tempo': 92.061, 'speechines': 0.0508, 'valence': 0.627}, '332fAqelC3Uik91QPxPcB8': {'danceability': 0.576, 'energy': 0.754, 'tempo': 94.737, 'speechines': 0.0497, 'valence': 0.663}, '6B

In [7]:
audio_features['13C7afV2SBHsR583SEktoP'] #Como No Queriendo

{'danceability': 0.608,
 'energy': 0.623,
 'tempo': 156.008,
 'speechines': 0.0298,
 'valence': 0.902}

In [13]:
tracks_data['6Vn1vje8yMw2OY6IOk9dkH']

{'artist_id': '790FomKkXshlbRYZFtlgla',
 'track_id': '6Vn1vje8yMw2OY6IOk9dkH',
 'track_name': 'GATÚBELA'}

In [9]:
index = audio_features.keys()
rows = [audio_features[k] for k in index]

df = pd.DataFrame(rows, index=index)
df.head()

,danceability,energy,tempo,speechines,valence
3GaGWoU1KfVqgl7vnRHP55,0.658,0.453,126.043,0.3900,0.358
4sDpxaulErbJ9NBaJDC7cJ,0.625,0.755,87.948,0.0724,0.429
7aJP1XcWcSU9w0eqSvaeZR,0.766,0.737,95.011,0.0634,0.964
7q3NthaBqFI6s0WnAlnIvq,0.819,0.596,165.951,0.3180,0.622
5ubqSAc9LxUS2CsMHcA4kF,0.632,0.750,91.170,0.0358,0.870


In [10]:
# using PERRO NEGRO as target track_id for finding similarity between original request and recommended songs
target_track_id = '7iQXYTyuG13aoeHxGG28Nh'

In [11]:
target_track_features = df.loc[target_track_id]

distances = scipy.spatial.distance.cdist(df, [target_track_features], metric="cosine")[:,0]

query_distances = list(zip(df.index, distances))

In [16]:
#Printing the top ten most similar actors to our target
for similar_track_id, similar_genre_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[1:11]:
    similar_actor = audio_features[similar_track_id]
    print(similar_track_id, tracks_data[similar_track_id]['track_name'],audio_features[similar_track_id], similar_genre_score)

6Vn1vje8yMw2OY6IOk9dkH GATÚBELA {'danceability': 0.853, 'energy': 0.859, 'tempo': 95.131, 'speechines': 0.306, 'valence': 0.325} 7.013821483603522e-07
2LWM5Ft0TdX24GeSq1u6PV FERXXO 30 {'danceability': 0.765, 'energy': 0.637, 'tempo': 89.968, 'speechines': 0.0807, 'valence': 0.508} 4.7861594054676715e-06
0zY4kXv0OOKLFh33DH2cSv Recuerdos {'danceability': 0.668, 'energy': 0.683, 'tempo': 78.982, 'speechines': 0.0643, 'valence': 0.471} 5.319228872036064e-06
4sDpxaulErbJ9NBaJDC7cJ Buenas {'danceability': 0.625, 'energy': 0.755, 'tempo': 87.948, 'speechines': 0.0724, 'valence': 0.429} 5.583428063782314e-06
0B8yu7yLe1UsXv6gewHRv2 Pintao {'danceability': 0.735, 'energy': 0.797, 'tempo': 105.765, 'speechines': 0.0638, 'valence': 0.471} 5.9973550498026995e-06
4waqcUQWdj0yH26STWl2Rq Rosas {'danceability': 0.783, 'energy': 0.685, 'tempo': 107.01, 'speechines': 0.0271, 'valence': 0.54} 7.906940536872575e-06
71nkNlFUY4d9mBvTAyV9ig Dónde Estás {'danceability': 0.706, 'energy': 0.623, 'tempo': 90.007,